In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

all_data = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

for page in range(1, 101):  # 100 صفحة
    url = f"https://www.goodreads.com/quotes?page={page}"
    
    response = requests.get(url, headers=headers, timeout=10)

    soup = BeautifulSoup(response.text, "html.parser")

    quotes = soup.find_all("div", class_="quote")

    for quote in quotes:
        text_element = quote.find("div", class_="quoteText")
        text = text_element.text.strip().split('―')[0].replace('"', '').strip() if text_element else "N/A"

        author_element = quote.find("span", class_="authorOrTitle")
        author = author_element.text.strip() if author_element else "N/A"

        book_element = quote.find("a", class_="authorOrTitle")
        book = book_element.text.strip() if book_element else "N/A"

        tags_elements = quote.find_all("a", class_="smallText")
        tags = [tag.text.strip() for tag in tags_elements if "likes" not in tag.text]

        likes_element = quote.find("a", class_="smallText", string=lambda x: "likes" in str(x))
        likes = likes_element.text.strip() if likes_element else "0 likes"

        all_data.append({
            "Quote": text[:300] + "..." if len(text) > 300 else text,
            "Author": author,
            "Book": book,
            "Tags": ", ".join(tags),
            "Likes": likes,
            "Page": page
        })

    
    time.sleep(random.uniform(1, 2))

df = pd.DataFrame(all_data)
df.to_csv("goodreads_quotes.csv", index=False, encoding="utf-8-sig")

